In [27]:
from models import *
from andi_datasets.models_phenom import models_phenom
from helpersGeneration import *
from helpersPlot import *
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2


# --------------- Example Usage --------------- #
# Define the embedding class and its arguments
from models import get_transformer_models

# Define hyperparameters
patch_size = 7
embed_dim = 64
num_heads = 4
hidden_dim = 128
num_layers = 6
dropout = 0.0

# Get all transformer models
models = get_transformer_models(patch_size, embed_dim, num_heads, hidden_dim, num_layers, dropout,name_suffix='_s')
models_big = get_transformer_models(patch_size, embed_dim*2, num_heads*2, hidden_dim*2, num_layers*2, dropout,name_suffix='_b')

models.update(models_big)
resnet = MultiImageLightResNet(patch_size)
models.update({"resnet": resnet})

print(models.keys())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
dict_keys(['linear_2layer_s', 'linear_1layer_s', 'cnn_1layer_s', 'deepcnn_1layer_s', 'linear_2layer_b', 'linear_1layer_b', 'cnn_1layer_b', 'deepcnn_1layer_b', 'resnet'])


In [12]:
# number of time steps per trajectory (frames)
T = 200
# number of trajectories
N = 2
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / s)
D = 5
nposframe = 10

trajs, labels = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajs.shape,labels.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajs = trajs.transpose(1,0,2)
labels = labels.transpose(1,0,2)
print(trajs.shape,labels.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labels[0,0,:]}")


# convert trajectories of D (pixels/s) to D (micro_m/ms)
trajs = trajs / 100


videos = trajectories_to_video(trajs,nposframe,
                                   center=True,
                                   image_props={"upsampling_factor":10,
                                            "background_intensity": [100,10],
                                            "particle_intensity": [500,20],
                                            "resolution": 100e-9,
                                            "trajectory_unit" : 1000,
                                            "output_size": 7,
                                            "poisson_noise" : -1
                                              })

print(videos.shape)
play_video(videos[0])
videos_t = torch.from_numpy(videos)
vid = videos_t[0]
print(vid.shape)

(200, 2, 2) (200, 2, 3)
(2, 200, 2) (2, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1. 5. 2.]
(2, 20, 7, 7)
vmin: 73.88610076904297 vmax: 521.956787109375 mean: 136.38


torch.Size([20, 7, 7])


In [29]:
for model_name in models.keys():
    model = models[model_name]
    out = model(videos_t)
    print(f"Prediction of: {model_name}: {out}")


Prediction of: linear_2layer_s: tensor([[-0.1274],
        [-0.1274]], grad_fn=<AddmmBackward0>)
Prediction of: linear_1layer_s: tensor([[-1.0376],
        [-1.0376]], grad_fn=<AddmmBackward0>)
Prediction of: cnn_1layer_s: tensor([[0.4198],
        [0.4198]], grad_fn=<AddmmBackward0>)
Prediction of: deepcnn_1layer_s: tensor([[-0.3254],
        [-0.3254]], grad_fn=<AddmmBackward0>)
Prediction of: linear_2layer_b: tensor([[0.3449],
        [0.3449]], grad_fn=<AddmmBackward0>)
Prediction of: linear_1layer_b: tensor([[-0.3770],
        [-0.3770]], grad_fn=<AddmmBackward0>)
Prediction of: cnn_1layer_b: tensor([[0.5125],
        [0.5125]], grad_fn=<AddmmBackward0>)
Prediction of: deepcnn_1layer_b: tensor([[-0.0208],
        [-0.0208]], grad_fn=<AddmmBackward0>)
Prediction of: resnet: tensor([[0.2448],
        [0.2537]], grad_fn=<MeanBackward1>)


In [30]:
def count_parameters(model):
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return num_params


for model_name in models.keys():
    model = models[model_name]
    num_p = count_parameters(model)
    print(f"Number of trainable parameters of {model_name}: {num_p}")


Number of trainable parameters of linear_2layer_s: 212673
Number of trainable parameters of linear_1layer_s: 204289
Number of trainable parameters of cnn_1layer_s: 204289
Number of trainable parameters of deepcnn_1layer_s: 302017
Number of trainable parameters of linear_2layer_b: 1629825
Number of trainable parameters of linear_1layer_b: 1596673
Number of trainable parameters of cnn_1layer_b: 1596673
Number of trainable parameters of deepcnn_1layer_b: 1699457
Number of trainable parameters of resnet: 315617
